In [2]:
import struct
import numpy

In [3]:
#Put all bytes from target file into memmap array
target_file = numpy.memmap(filename = "tests/samples/sample-6.14.00-uncompressed.root", mode="r", dtype = numpy.uint8, shape = (80618,))

In [6]:
_format1       = ">4si"
_format2_small = ">iiiiiiBiii18s"
_format2_big   = ">iqqiiiBiqi18s"
_format3       = ">hIIii"
_format4_small = ">iii"
_format4_big   = ">qqq"
_format5       = ">i"

In [7]:
name = input("Enter name of the ROOT file - ")
if name.endswith(".root") is False:
    name = name + ".root"
file = numpy.memmap(filename = name, dtype = numpy.uint8, mode = "w+", shape = (80618,))
#In the case of a 1D array, the shape is the size of the file
#In ROOT files, the fEND corresponds to the size of the file
#fEND for ~/Downloads/uproot/tests/samples/sample-6.14.00-uncompressed.root is 80618

In [8]:
#Header bytes
fVersion = 61400
toadd = numpy.frombuffer(struct.pack(_format1, b"root", fVersion), dtype=numpy.uint8)
count = len(toadd)
file[:count] = toadd
#constants for file sample-6.14.00-uncompressed.root
fBEGIN = 100
fEND = 80618
fSeekFree = 80542
fNbytesFree = 76
nfree = 1
fNbytesName = 100
fUnits = 4
fCompress = 100
fSeekInfo = 63150
fNbytesInfo = 17282
fUUID = b"\x00\x01\x11\xaf/\xd0t^\x11\xe8\x8a\x9b\x01\x00\x00\x7f\xbe\xef"
toadd = numpy.frombuffer(struct.pack(_format2_small, fBEGIN, fEND, fSeekFree, fNbytesFree, nfree, fNbytesName, fUnits, fCompress, fSeekInfo, fNbytesInfo, fUUID), dtype=numpy.uint8)
file[count:(count + len(toadd))] = toadd
count = count + len(toadd)

In [ ]:
#Temporary position
fVersion = 5
fDatimeC = 1571330984
fNbytesKeys = 110
fNbytesName = 100
fDatimeM = 1571330985
toadd = numpy.frombuffer(struct.pack(_format3, fVersion, fDatimeC, fDatimeM, fNbytesKeys, fNbytesName), dtype=numpy.uint8)
file[count:(count + len(toadd))] = toadd
count = count + len(toadd)

In [9]:
#Hexdump examining block
target_file[:count]

memmap([114, 111, 111, 116,   0,   0, 239, 216,   0,   0,   0, 100,   0,
          1,  58, 234,   0,   1,  58, 158,   0,   0,   0,  76,   0,   0,
          0,   1,   0,   0,   0, 100,   4,   0,   0,   0, 100,   0,   0,
        246, 174,   0,   0,  67, 130,   0,   1,  17, 175,  47, 208, 116,
         94,  17, 232, 138, 155,   1,   0,   0, 127, 190, 239],
       dtype=uint8)

In [ ]:
#Test - See bytes read of target file
!hexdump -C tests/samples/sample-6.14.00-uncompressed.root | head

In [ ]:
#Test - See bytes read of copied file
!hexdump -C rootpy.root | head

In [ ]:
#Test - uproot is reading it correctly
import uproot
uproot.open('~/Downloads/uproot/rootpy.root')

In [ ]:
#Test - Check size of file
import os
statinfo_target = os.stat('tests/samples/sample-6.14.00-uncompressed.root')
print ("Size of target file is ", statinfo_target.st_size)
statinfo_copy = os.stat('rootpy.root')
print ("Size of written file is ", statinfo_copy.st_size)